# Valuing Cross Currency Basis Swaps (fixed to float)

This notebook demonstrates how to perform cross currency swap (fixed to float) pricing within LSEG Financial Analytics SDK.

## Imports

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

The following modules will be required in this notebook:

In [ ]:
import pandas as pd
import json
from lseg_analytics.pricing.templates.instrument_templates import search, load 
from lseg_analytics.pricing.instruments.ir_swaps import load, Rate, UnitEnum
from lseg_analytics.pricing.instruments.ir_swaps import value, IrPricingParameters, IrSwapDefinitionInstrument
from lseg_analytics.pricing.instruments.ir_swaps import solve, IrPricingParameters, IrSwapSolvingParameters, IrSwapSolvingTarget, IrSwapSolvingVariable, IrMeasure, IrSwapDefinitionInstrument
from lseg_analytics.pricing.instruments.ir_swaps import create_from_ccs_template, CrossCurencySwapOverride
from lseg_analytics.pricing.common import PaidLegEnum, ReferenceDate, RelativeAdjustableDate
from datetime import date
from IPython.display import display

## Utility methods

These methods are used throughout the notebook to display results.

In [3]:
def extract_tag_key(tag):
    return tag.split(":")[0] if ":" in tag else tag

def list_unique_tags(all_swaps):
    unique_tags = set()
    for item in all_swaps:
        tags = item.get("description", {}).get("tags", [])
        for tag in tags:
            key = extract_tag_key(tag)
            unique_tags.add(key)
    return unique_tags

def display_templates(templates):
    unique_tag_keys = list(list_unique_tags(templates))

    rows = []
    for item in templates:
        row = {
            "Space": item.get("location", {}).get("space", ""),
            "Id": item.get("id", ""),
            "Name": item.get("location", {}).get("name", ""),
            "Summary": item.get("description", {}).get("summary", ""),
        }
        tags = item.get("description", {}).get("tags", [])
        tag_dict = {extract_tag_key(tag): tag for tag in tags}
        for key in unique_tag_keys:
            tag_val = tag_dict.get(key, None)
            if tag_val is not None and ":" in tag_val:
                row[key] = tag_val.split(":", 1)[1]
            else:
                row[key] = tag_val
        rows.append(row)

    display(pd.DataFrame(rows))

def build_legs_comparison_df(first_leg, second_leg):
    """Builds a DataFrame comparing key fields of first_leg and second_leg."""
    def get_nested_attr(obj, attrs, default=None):
        for attr in attrs:
            obj = getattr(obj, attr, None)
            if obj is None:
                return default
        return obj

    rows = [
        {
            "Field": "description",
            "First Leg": get_nested_attr(first_leg, ["description", "leg_description"]),
            "Second Leg": get_nested_attr(second_leg, ["description", "leg_description"]),
        },
        {
            "Field": "market value",
            "First Leg": get_nested_attr(first_leg, ["valuation", "market_value", "deal_currency", "value"]),
            "Second Leg": get_nested_attr(second_leg, ["valuation", "market_value", "deal_currency", "value"]),
        },
        {
            "Field": "DV01",
            "First Leg": get_nested_attr(first_leg, ["risk", "dv01", "value"]),
            "Second Leg": get_nested_attr(second_leg, ["risk", "dv01", "value"]),
        },
    ]

    display(pd.DataFrame(rows))


## Template-based setup for swap valuation

Using an existing swap template to build a swap, user can adjust it with his own settings.
- Swap conventions provide a standardized set of market-accepted terms, such as leg frequencies and interest calculation methods.
- By referencing a swap convention, you ensure alignment with market standards while streamlining the setup process.

In [3]:
#search function to find fix to float cross currency swap templates
fixed_to_float_templates = search(
    item_per_page= 5,
    tags=["instrumentType:CrossCurrencySwap"],
    spaces=["LSEG"])

display_templates(fixed_to_float_templates)


,Space,Id,Name,Summary,instrumentType,currency,index
0,LSEG,88273d49-7f81-4972-af14-83cb6e64de97,CNHUSQMSRCS,US Dollar OIS SOFR vs Chinese 3-Month Hibor Ba...,CrossCurrencySwap,USD,USD_SOFR_ON
1,LSEG,0b9641da-3148-4849-affa-d72b62aa9dd8,CNUSQMSRBS,US Dollar OIS SOFR vs Chinese 3-Month Shibor B...,CrossCurrencySwap,USD,USD_SOFR_ON
2,LSEG,be6e21aa-51ae-4dbd-bde1-1d4ab23d3c5a,MXNU6L,US dollar 6-Month Libor vs Mexican peso (MXV) ...,CrossCurrencySwap,USD,USD_LIBOR_6M
3,LSEG,6c1680cb-9536-46b1-b463-bd8944e165a9,MXNUDIT,Mexican peso (MXV) fixed 182d UDI vs Mexican p...,CrossCurrencySwap,MXN,MXN_TIIE_28D
4,LSEG,282cf7f9-e16d-40cf-81c6-d6f4e78a3157,PEUSNDSMSRBS,US Dollar OIS SOFR vs Peruvian sol fixed 1 yea...,CrossCurrencySwap,USD,USD_SOFR_ON


## Editing the swap template to fit our requirements

- When pricing a specific swap, user often needs to tailor certain parameters—like start date, end date, or fixed rate to suit requirements.
- The framework allows user to override only the necessary terms, combining efficiency with full customization for transaction.

In [4]:
swap_template = "LSEG/CNHUSQMSRCS"

# define a 1 year forward start date for swap
start_date = RelativeAdjustableDate(tenor = "1Y", reference_date= ReferenceDate.VALUATION_DATE)

# setup a 2Y swap maturity from the start date
end_date = RelativeAdjustableDate(tenor = "2Y", reference_date= ReferenceDate.START_DATE)

# override the swap template with a 1M USD, notional, the previous start_date and end_date, and a 10bp spread on the float leg
par_overrides = CrossCurencySwapOverride(
    start_date = start_date,
    end_date=end_date,
    amount=7200000,
    contra_amount=1000000,
    spread = Rate(value=10.0, unit = UnitEnum.BASIS_POINT),
    paid_leg= PaidLegEnum.FIRST_LEG)

# build the swap from 'LSEG/CNUSQMSRBS' template and previously defined overrides
fwd_start_ccs = create_from_ccs_template(template_reference = swap_template, overrides = par_overrides)

fwd_start_ccs_def = IrSwapDefinitionInstrument(definition = fwd_start_ccs.definition)

Output of the created cross currency swap definition can be presented as follows:

In [5]:
print(json.dumps(fwd_start_ccs.definition.as_dict(), indent=4))

{
    "firstLeg": {
        "rate": {
            "interestRateType": "FixedRate",
            "rate": {
                "value": 0,
                "unit": "Percentage"
            }
        },
        "interestPeriods": {
            "startDate": {
                "dateType": "RelativeAdjustableDate",
                "tenor": "1Y",
                "referenceDate": "ValuationDate"
            },
            "endDate": {
                "dateType": "RelativeAdjustableDate",
                "tenor": "2Y",
                "referenceDate": "StartDate"
            },
            "frequency": "Quarterly",
            "businessDayAdjustment": {
                "calendars": [
                    "CHN",
                    "USA"
                ],
                "convention": "ModifiedFollowing"
            },
            "rollConvention": "Same"
        },
        "paymentOffset": {
            "tenor": "2D",
            "businessDayAdjustment": {
                "calendars": [
             

## Solving for a forward fixed to float cross currency swap fixed rate

The par swap rate for prepared forward starting cross currency swap.

In [6]:
# define solving parameters in order to compute the par fixed rate
solving_params = IrSwapSolvingParameters(
    target=IrSwapSolvingTarget(market_value=IrMeasure(value=0.0)),
    variable= IrSwapSolvingVariable(leg = "FirstLeg", name = "FixedRate"))

# define a solving date
solving_date = date(2025, 5, 10)

# solve the swap fixed rate
valuation_response = solve(
    definitions=[fwd_start_ccs_def],
    pricing_preferences= IrPricingParameters(
        solving_parameters = solving_params,
        valuation_date=solving_date
    )
)

# print the solved spread
solved_rate = valuation_response.analytics[0].solving.result

print(
    "\t Solved fixed rate, in %:", solved_rate,
)

build_legs_comparison_df(valuation_response.analytics[0].first_leg, valuation_response.analytics[0].second_leg)


	 Solved fixed rate, in %: 0.2442914345336445


,Field,First Leg,Second Leg
0,description,Pay CNY Quarterly 0.24%,Receive USD Quarterly +10bp SOFR
1,market value,6966132.732305,6966132.732305
2,DV01,2070.024775,678.549244


## Saving the solved swap

In [7]:
instrument_id = "CNY_SOFR_1Y2Y"

# create a 'booked_trade' updated with the solved fixed rate
trade_overrides = CrossCurencySwapOverride(
    start_date = start_date,
    end_date=end_date,
    amount=7200000,
    contra_amount=1000000,
    fixed_rate= Rate(value=solved_rate, unit = UnitEnum.PERCENTAGE),
    spread = Rate(value=10.0, unit = UnitEnum.BASIS_POINT),
    paid_leg= PaidLegEnum.FIRST_LEG)

# create the swap using the selected swap template
booked_trade = create_from_ccs_template(template_reference = swap_template, overrides = trade_overrides)

try:
    # Check if the instrument already exists in HOME space
    booked_trade = load(name=instrument_id, space="HOME")
    print(f"Instrument {instrument_id} already exists in HOME space.")
except:
    # If the instrument does not exist in HOME space, we can save it
    booked_trade.save(name=instrument_id, space="HOME")
    print(f"Instrument {instrument_id} saved in HOME space.")

[ERROR]	[2025-05-22 18:23:21,117]	[MainThread]	[lseg_analytics.instruments.ir_swaps]	[_functions.py:276]	 IrSwap CNY_SOFR_1Y2Y not found in space='HOME'
Instrument CNY_SOFR_1Y2Y saved in HOME space.


# Revalue the trade post booking

In [8]:
definition = IrSwapDefinitionInstrument(definition = booked_trade.definition)

# define a mark to market date as of 2025-05-15
mtm_date = date(2025, 5, 15)

# value the booked swap
valuation_response = value(
    definitions=[definition],
    pricing_preferences= IrPricingParameters(valuation_date=mtm_date, report_currency="USD")
)

# print mark to market results
print(
    "\t MtM in USD:", valuation_response.analytics[0].valuation.market_value.report_currency.value,
)

build_legs_comparison_df(valuation_response.analytics[0].first_leg, valuation_response.analytics[0].second_leg)


	 MtM in USD: -3437.7801879328


,Field,First Leg,Second Leg
0,description,Pay CNY Quarterly 0.24%,Receive USD Quarterly +10bp SOFR
1,market value,6958699.620765,6933922.507617
2,DV01,2061.367348,669.525148


## Deleting the saved instrument

In [ ]:
# Delete the instrument we created in HOME space
from lseg_analytics.pricing.instruments.ir_swaps import delete

delete(name=instrument_id, space="HOME")

True